# Idea
Do some training

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import pickle

In [2]:
import sys
repo_base_path = Path("./../").resolve()
assert str(repo_base_path).endswith("csg_is"), f"{repo_base_path} is not a valid path to the CSG_IS repository" 

sys.path.append(str(repo_base_path))

# from py_dataset import get_all_files_df
# from py_dataset import read_in_files
from py_dataset import feature_plotting
# from py_dataset import feature_selection
# from py_dataset import net_feature_engineering
# from py_dataset import entropy_feature_engineering
# from py_dataset.classes import DataSource
from py_dataset import sys_func



In [3]:
df = pd.read_csv('all_df.csv')
df.set_index("timestamp", inplace=True)

In [4]:
df["label"].unique()

array(['1_normal', nan, '2_ransomware', '3_thetick', '4_bashlite',
       '5_httpbackdoor', '6_beurk', '7_backdoor', '8_bdvl', '9_xmrig'],
      dtype=object)

### Preprocessings
- remove Null values
- drop timestamp
- labelEncode
- normalize

In [5]:
print(df.shape)
old_df = df.copy()
df.dropna(axis=0, how='any', inplace=True)

print(df.shape)

(43248, 306)
(25770, 306)


In [6]:
df.reset_index(inplace=True, drop=True)

In [7]:
label_encoder = sys_func.create_label_encoder_for_labels()
df['label'] = label_encoder.transform(df['label'])

In [8]:
df = feature_plotting.test_transform_all_to_numeric_columns(df)

In [9]:
cols_to_exclude_from_scaling = ["label"]
vectors = feature_plotting.minmax_scale_features(df,remove_outliers_iqr_all_columns=False,
                                                        cols_to_exclude_from_scaling=cols_to_exclude_from_scaling)
# print(vectors.shape)
# print(" with dropped null: ", vectors.dropna(axis=0, how='any', inplace=False).shape)

In [10]:
df["label"].unique(), vectors["label"].unique()

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([0, 1, 2, 3, 4, 5, 6, 7, 8]))

### Setup

In [11]:
from torch.utils.data import Dataset, DataLoader
import torch

class CustomeDataset(Dataset):
    def __init__(self, some_df:pd.DataFrame):
        
        self.x = torch.tensor(some_df.drop(columns=["label"], inplace=False).values, dtype=torch.float32)
        self.labels = torch.tensor(some_df["label"].values, dtype=torch.uint8)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()
        # print(self.x[idx], self.labels[idx])
        return self.x[idx], self.labels[idx]

In [17]:
from mlp import MalwaresModelMLP
from torch.utils.data import DataLoader
import lightning
from sklearn.model_selection import train_test_split
from lightning import loggers as pl_loggers
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint

# tensorboard = pl_loggers.TensorBoardLogger('./')

# split into train and val
train, val = train_test_split(vectors, test_size=0.2)

train_dataset = CustomeDataset(train)
train_dataloader = DataLoader(train_dataset, batch_size=500, shuffle=True, num_workers=7)

val_dataset = CustomeDataset(val)
val_dataloader = DataLoader(val_dataset, batch_size=500, shuffle=False, num_workers=7)

# trainer = lightning.Trainer(fast_dev_run=100)
monitor = "Validation/Accuracy"
early_stop_callback = EarlyStopping(monitor=monitor, patience=5, verbose=True, mode="max")
checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor=monitor, mode="max")

trainer = lightning.Trainer(max_epochs=150, callbacks=[early_stop_callback, checkpoint_callback], enable_checkpointing=True) #logger=None, 


model = MalwaresModelMLP(input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [18]:
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                      | Params
------------------------------------------------------------
0 | train_metrics | MetricCollection          | 0     
1 | val_metrics   | MetricCollection          | 0     
2 | test_metrics  | MetricCollection          | 0     
3 | cm            | MulticlassConfusionMatrix | 0     
4 | criterion     | CrossEntropyLoss          | 0     
5 | l1            | Linear                    | 9.2 K 
6 | l2            | Linear                    | 930   
7 | l3            | Linear                    | 279   
------------------------------------------------------------
10.4 K    Trainable params
0         Non-trainable params
10.4 K    Total params
0.042     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 0
ValidationEpoch/CM
 tensor([[  0,   0,   0,   0,   0,   0,   0,   0, 101],
        [  0,   0,   0,   0,   0,   0,   1,   0, 117],
        [  0,   0,   0,   0,   0,   0,   0,   0, 124],
        [  0,   0,   0,   0,   0,   0,   0,   0, 105],
        [  0,   0,   0,   0,   0,   0,   0,   0,  98],
        [  0,   0,   0,   0,   0,   0,   0,   0, 113],
        [  0,   0,   0,   0,   0,   0,   0,   0,  94],
        [  0,   0,   0,   0,   0,   0,   2,   0, 119],
        [  0,   0,   0,   0,   0,   0,   0,   0, 126]])


/home/<User>/repos/csg_is/.venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (42) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Epoch end: Validation, epoch number: 27
ValidationEpoch/CM
 tensor([[61423,   140,   470,  4576,  1320,     1,   862,     0,   159],
        [    0, 88490,     0,     0,     0,     0,     2,     1,   185],
        [  200,   213, 74947,   272,   124,   292,  9767,     0,   169],
        [ 6857,   155,   681, 59650,  1380,     2,  2765,     0,   165],
        [  835,   133,   548,   758, 70596,    29,   353,     0,   151],
        [    0,   179,   276,     0,     0, 75610,    13,     0,   175],
        [ 1231,   166, 11688,  5233,   362,   212, 52608,     0,   144],
        [    0,   137,     0,     0,     0,     0,     3, 71237,   159],
        [    0,   122,    13,     0,     0,     0,     0,     0, 88551]])
Epoch end: Train, epoch number: 26
TrainEpoch/CM
 tensor([[61423,   140,   470,  4576,  1320,     1,   862,     0,   159],
        [    0, 88490,     0,     0,     0,     0,     2,     1,   185],
        [  200,   213, 74947,   272,   124,   292,  9767,     0,   169],
        [ 685

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 28
ValidationEpoch/CM
 tensor([[63961,   140,   470,  4586,  1320,     1,   864,     0,   159],
        [    0, 91769,     0,     0,     0,     0,     2,     2,   185],
        [  200,   213, 77973,   272,   124,   297,  9916,     0,   169],
        [ 6863,   155,   682, 62234,  1380,     2,  2824,     0,   165],
        [  835,   133,   548,   762, 73304,    29,   355,     1,   151],
        [    0,   182,   276,     0,     0, 78427,    13,     0,   175],
        [ 1235,   166, 11861,  5439,   362,   218, 54869,     0,   144],
        [    0,   137,     0,     0,     0,     0,     3, 73882,   159],
        [    0,   122,    14,     0,     0,     0,     0,     0, 91830]])


Monitored metric Validation/Accuracy did not improve in the last 5 records. Best score: 0.985. Signaling Trainer to stop.


Epoch end: Train, epoch number: 27
TrainEpoch/CM
 tensor([[63961,   140,   470,  4586,  1320,     1,   864,     0,   159],
        [    0, 91769,     0,     0,     0,     0,     2,     2,   185],
        [  200,   213, 77973,   272,   124,   297,  9916,     0,   169],
        [ 6863,   155,   682, 62234,  1380,     2,  2824,     0,   165],
        [  835,   133,   548,   762, 73304,    29,   355,     1,   151],
        [    0,   182,   276,     0,     0, 78427,    13,     0,   175],
        [ 1235,   166, 11861,  5439,   362,   218, 54869,     0,   144],
        [    0,   137,     0,     0,     0,     0,     3, 73882,   159],
        [    0,   122,    14,     0,     0,     0,     0,     0, 91830]])


In [14]:
model.eval()

MalwaresModelMLP(
  (train_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=Train/
  )
  (val_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=Validation/
  )
  (test_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=Test/
  )
  (cm): MulticlassConfusionMatrix()
  (criterion): CrossEntropyLoss()
  (l1): Linear(in_features=305, out_features=30, bias=True)
  (l2): Linear(in_features=30, out_features=30, bias=True)
  (l3): Linear(in_features=30, out_features=9, bias=True)
)

In [23]:
print(checkpoint_callback.best_model_path)   # prints path to the best model's checkpoint
print(checkpoint_callback.best_model_score) # and prints it score
best_model = MalwaresModelMLP.load_from_checkpoint(checkpoint_callback.best_model_path, input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_))

trainer.test(best_model, dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/home/<User>/repos/csg_is/training/lightning_logs/version_7/checkpoints/epoch=22-step=966.ckpt
tensor(0.9851, device='cuda:0')


Testing: |          | 0/? [00:00<?, ?it/s]

Epoch end: Test, epoch number: 0
TestEpoch/CM
 tensor([[511,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0, 673,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0, 618,   0,   0,   0,  31,   0,   0],
        [  0,   0,   0, 519,   0,   0,   7,   0,   0],
        [  0,   0,   0,   0, 512,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0, 588,   0,   0,   0],
        [  0,   0,  18,  18,   0,   0, 460,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0, 550,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0, 649]])
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Test/Accuracy         0.9850842952728271
      Test/F1Score          0.9847362637519836
        Test/Loss          0.042185284197330475
     Test/Precision         0

[{'Test/Loss': 0.042185284197330475,
  'Test/Accuracy': 0.9850842952728271,
  'Test/Precision': 0.9848151803016663,
  'Test/Recall': 0.9850842952728271,
  'Test/F1Score': 0.9847362637519836,
  'TestEpoch/Accuracy': 0.9851495027542114,
  'TestEpoch/Precision': 0.984652578830719,
  'TestEpoch/Recall': 0.9851495027542114,
  'TestEpoch/F1Score': 0.9848780632019043}]